In [9]:
import os
import numpy as np
import pandas as pd
import pickle
import joblib
import matplotlib
import matplotlib.pyplot as plt
import shap
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from matplotlib.colors import LinearSegmentedColormap, ListedColormap
import warnings
import time
from datetime import datetime
warnings.filterwarnings('ignore')

# ====================== 0. 自定义配色方案 ======================
# 使用与原SHAP程序相同的9种颜色创建自定义配色方案
custom_colors = [
    (142/255, 1/255, 82/255),      # 紫红色
    (201/255, 44/255, 134/255),    # 深粉色
    (229/255, 144/255, 191/255),   # 中粉色
    (248/255, 207/255, 231/255),   # 浅粉色
    (248/255, 242/255, 245/255),   # 浅粉白色
    (229/255, 245/255, 206/255),   # 极浅绿色
    (170/255, 216/255, 117/255),   # 浅绿色
    (104/255, 169/255, 50/255),    # 中绿色
    (52/255, 116/255, 27/255)      # 深绿色
]

# 创建颜色映射
custom_cmap_full = ListedColormap(custom_colors)
custom_cmap = LinearSegmentedColormap.from_list("custom_diverging", custom_colors, N=256)

# ====================== 1. 定义路径和参数 ======================
# 输入路径（基于第一个程序的输出）
model_dir = r"D:\博一下\manuscript5-变Si组\模型训练_K45_物理引导特征工程_无贝叶斯优化-7.29.3最终_修复版"
data_file = r"D:\文成数据库\Nb-Si数据库3.4-成分-性能.xlsx"

# 输出路径（用户指定的保存地址）
shap_output_dir = r"D:\博一下\manuscript5-变Si组\SHAP分析8.1.3"

# 创建SHAP输出目录
if not os.path.exists(shap_output_dir):
    os.makedirs(shap_output_dir)

print("=== 开始多模型SHAP分析（修复版）===")
print(f"输出目录: {shap_output_dir}")
print(f"分析时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# ====================== 2. 加载原始数据 ======================
print("\n=== 加载原始数据 ===")
df = pd.read_excel(data_file)
print(f"✓ 成功加载数据: {df.shape}")

# ====================== 3. 加载模型和相关数据 ======================
print("\n=== 加载所有最佳模型相关数据 ===")

try:
    # 加载最佳模型列表
    models_file = os.path.join(model_dir, 'best_models_physics_guided_no_bayesian_fixed.pkl')
    with open(models_file, 'rb') as f:
        best_models = pickle.load(f)
    
    # 加载特征名称列表
    features_file = os.path.join(model_dir, 'features_name_physics_guided_no_bayesian_fixed.pkl')
    with open(features_file, 'rb') as f:
        all_features_name = pickle.load(f)
    
    # 加载标准化器
    scaler_file = os.path.join(model_dir, 'scaler_physics_guided_no_bayesian_fixed.pkl')
    scaler = joblib.load(scaler_file)
    
    # 加载数据信息
    data_file_path = os.path.join(model_dir, 'data_info_physics_guided_no_bayesian_fixed.pkl')
    with open(data_file_path, 'rb') as f:
        data_info = pickle.load(f)
    
    print("✓ 成功加载所有保存的数据")
    print(f"✓ 找到 {len(best_models)} 个最佳模型")
    
except Exception as e:
    print(f"❌ 加载保存数据失败: {e}")
    raise

# ====================== 4. 数据预处理函数 ======================
def prepare_shap_data_for_model(model_info, all_features_name, df, data_info, scaler):
    """
    为特定模型准备SHAP分析数据
    """
    print(f"  准备模型数据: {model_info['model_key']}")
    
    # 获取模型的特征
    features = model_info['features']
    sel_idx = model_info['sel_idx']
    
    print(f"    特征: {features}")
    print(f"    特征索引: {sel_idx}")
    
    try:
        # 识别哪些是原始特征，哪些是物理工程特征
        original_excel_features = df.columns.tolist()
        
        # 分类特征
        original_features = []      # 存在于原始Excel中的特征
        physics_features = []       # 物理工程生成的特征
        original_indices = []       # 原始特征在all_features_name中的索引
        physics_indices = []        # 物理特征在all_features_name中的索引
        
        for i, feature in enumerate(all_features_name):
            if feature in original_excel_features:
                original_features.append(feature)
                original_indices.append(i)
            else:
                physics_features.append(feature)
                physics_indices.append(i)
        
        # 1. 从原始Excel文件提取原始特征
        if len(original_features) > 0:
            X_original_from_excel = df[original_features].values
            y_from_excel = df['KQ'].values
            
            # 移除NaN值
            nan_mask_x_orig = ~np.isnan(X_original_from_excel).any(axis=1)
            nan_mask_y = ~np.isnan(y_from_excel)
            valid_mask_excel = nan_mask_x_orig & nan_mask_y
            
            X_original_clean = X_original_from_excel[valid_mask_excel]
            y_all_clean = y_from_excel[valid_mask_excel]
        else:
            X_original_clean = None
            y_all_clean = data_info['y_all_np']
        
        # 2. 从保存的训练数据提取物理工程特征
        if len(physics_features) > 0:
            # 根据模型类型选择合适的数据源
            if model_info['is_tree_based']:
                saved_data = data_info['X_all_original']
            else:
                saved_data = data_info['X_all_scaled']
            
            # 提取物理特征
            X_physics_from_saved = saved_data[:, physics_indices]
            
            # 确保样本数量一致
            if X_original_clean is not None:
                min_samples = min(X_original_clean.shape[0], X_physics_from_saved.shape[0])
                X_original_clean = X_original_clean[:min_samples]
                X_physics_from_saved = X_physics_from_saved[:min_samples]
                y_all_clean = y_all_clean[:min_samples]
        else:
            X_physics_from_saved = None
        
        # 3. 合并数据并保持正确的特征顺序
        if X_original_clean is not None and X_physics_from_saved is not None:
            # 创建完整的特征矩阵，按照all_features_name的顺序
            X_all_clean = np.zeros((X_original_clean.shape[0], len(all_features_name)))
            
            # 填入原始特征
            for i, orig_idx in enumerate(original_indices):
                X_all_clean[:, orig_idx] = X_original_clean[:, i]
            
            # 填入物理特征
            for i, phys_idx in enumerate(physics_indices):
                X_all_clean[:, phys_idx] = X_physics_from_saved[:, i]
            
        elif X_original_clean is not None:
            X_all_clean = X_original_clean
        elif X_physics_from_saved is not None:
            X_all_clean = X_physics_from_saved
        else:
            raise ValueError("无法获取任何特征数据")
        
        # 验证数据完整性
        if np.any(np.isnan(X_all_clean)):
            nan_mask = ~np.isnan(X_all_clean).any(axis=1)
            X_all_clean = X_all_clean[nan_mask]
            y_all_clean = y_all_clean[nan_mask]
        
    except Exception as e:
        print(f"    智能数据提取失败: {e}")
        print("    回退到保存数据方案...")
        
        # 回退方案：完全使用保存的训练数据
        if model_info['is_tree_based']:
            X_all_clean = data_info['X_all_original']
        else:
            X_all_clean = data_info['X_all_scaled'] 
        
        y_all_clean = data_info['y_all_np']
    
    # 选择模型使用的特征
    if model_info['is_tree_based']:
        # 树模型使用原始数据
        X_model = X_all_clean[:, sel_idx]
        final_data_type = "原始数据"
    else:
        # 其他模型使用标准化数据
        X_all_clean = scaler.transform(X_all_clean)
        X_model = X_all_clean[:, sel_idx]
        final_data_type = "标准化数据"
    
    print(f"    模型输入数据形状: {X_model.shape}")
    print(f"    使用数据类型: {final_data_type}")
    
    # 验证模型预测
    try:
        model = model_info['model_instance']
        predictions = model.predict(X_model)
        print(f"    模型预测成功: [{predictions.min():.4f}, {predictions.max():.4f}]")
        
        # 确保数据类型正确
        X_model = X_model.astype(np.float64)
        
        # 检查是否存在极值
        if np.any(np.isinf(X_model)) or np.any(np.isnan(X_model)):
            finite_mask = np.all(np.isfinite(X_model), axis=1)
            X_model = X_model[finite_mask]
            y_all_clean = y_all_clean[finite_mask]
            print(f"    清理后数据形状: X{X_model.shape}, y{y_all_clean.shape}")
        
        return X_model, y_all_clean, model, features, final_data_type
        
    except Exception as e:
        print(f"    模型预测失败: {e}")
        raise

# ====================== 5. SHAP可视化函数（完整版本）======================
def create_shap_visualizations(shap_values, X_model, features, model_output_dir, model_key, explainer, model, y_all_clean):
    """
    创建完整的SHAP可视化图表（与原版程序相同）
    """
    print(f"    创建完整SHAP可视化图表...")
    
    # 设置matplotlib参数
    plt.rcParams['figure.dpi'] = 300
    plt.rcParams['savefig.dpi'] = 300
    try:
        plt.rcParams['font.sans-serif'] = ['SimHei']
        plt.rcParams['axes.unicode_minus'] = False
    except:
        pass
    
    visualization_files = []
    
    # 5.1 SHAP概要图 (Summary Plot)
    try:
        plt.figure(figsize=(14, 10))
        
        shap.summary_plot(
            shap_values, 
            X_model, 
            feature_names=features, 
            show=False, 
            max_display=len(features),
            cmap=custom_cmap,
            alpha=0.8
        )
        
        plt.title(f'SHAP 特征影响概要图 - {model_key}', fontsize=16, fontweight='bold')
        plt.xlabel('SHAP值 (特征对输出的影响)', fontsize=12)
        plt.grid(axis='x', linestyle='--', alpha=0.3)
        plt.axvline(x=0, color='gray', linestyle='-', alpha=0.3)
        
        plt.figtext(0.13, 0.02, 
                    "说明: 紫色系表示特征值低→结果减小，绿色系表示特征值高→结果增大", 
                    fontsize=10, color='dimgray')
        
        plt.tight_layout()
        summary_file = os.path.join(model_output_dir, 'shap_summary_plot.png')
        plt.savefig(summary_file, dpi=300, bbox_inches='tight')
        plt.close()
        visualization_files.append(summary_file)
        print(f"      ✓ SHAP概要图")
    except Exception as e:
        print(f"      ❌ SHAP概要图失败: {e}")
    
    # 5.2 SHAP蜂群图 (Beeswarm Plot)
    try:
        plt.figure(figsize=(14, 10))
        
        shap.summary_plot(
            shap_values, 
            X_model, 
            feature_names=features, 
            show=False, 
            max_display=len(features),
            plot_type="dot",
            cmap=custom_cmap,
            alpha=0.8
        )
        
        plt.title(f'SHAP 蜂群图 - {model_key}', fontsize=16, fontweight='bold')
        plt.xlabel('SHAP值 (特征对输出的影响)', fontsize=12)
        plt.grid(axis='x', linestyle='--', alpha=0.3)
        plt.axvline(x=0, color='gray', linestyle='-', alpha=0.3)
        
        plt.tight_layout()
        beeswarm_file = os.path.join(model_output_dir, 'shap_beeswarm_plot.png')
        plt.savefig(beeswarm_file, dpi=300, bbox_inches='tight')
        plt.close()
        visualization_files.append(beeswarm_file)
        print(f"      ✓ SHAP蜂群图")
    except Exception as e:
        print(f"      ❌ SHAP蜂群图失败: {e}")
    
    # 5.3 SHAP条形图 (Bar Plot)
    try:
        plt.figure(figsize=(12, 8))
        
        shap.summary_plot(
            shap_values, 
            X_model, 
            feature_names=features, 
            plot_type='bar', 
            show=False, 
            max_display=len(features),
            color=custom_colors[8]
        )
        
        plt.title(f'SHAP 特征重要性条形图 - {model_key}', fontsize=16, fontweight='bold')
        plt.xlabel('平均|SHAP值|', fontsize=12)
        plt.tight_layout()
        bar_file = os.path.join(model_output_dir, 'shap_bar_plot.png')
        plt.savefig(bar_file, dpi=300, bbox_inches='tight')
        plt.close()
        visualization_files.append(bar_file)
        print(f"      ✓ SHAP条形图")
    except Exception as e:
        print(f"      ❌ SHAP条形图失败: {e}")
    
    # 5.4 SHAP热图
    try:
        # 计算特征重要性排序
        mean_abs_shap = np.mean(np.abs(shap_values), axis=0)
        importance_order = np.argsort(mean_abs_shap)[::-1]
        
        # 限制样本数量以防止图像过大
        max_samples_heatmap = min(200, shap_values.shape[0])
        sample_indices = np.random.choice(shap_values.shape[0], max_samples_heatmap, replace=False)
        
        # 按重要性排序重新排列数据
        ordered_shap_values = shap_values[sample_indices][:, importance_order]
        ordered_feature_names = [features[i] for i in importance_order]
        
        plt.figure(figsize=(16, 12))
        
        # 创建热图
        im = plt.imshow(ordered_shap_values.T, aspect='auto', cmap=custom_cmap, interpolation='none')
        
        # 添加颜色条
        cbar = plt.colorbar(im, label='SHAP值')
        cbar.ax.tick_params(labelsize=10)
        
        # 设置标题和标签
        plt.title(f'SHAP 热图 - {model_key} (按特征重要性排序)', fontsize=16, fontweight='bold')
        plt.xlabel('样本', fontsize=12)
        plt.ylabel('特征 (按重要性排序)', fontsize=12)
        
        # 设置y轴刻度为特征名称
        plt.yticks(range(len(ordered_feature_names)), ordered_feature_names, fontsize=10)
        plt.xticks(fontsize=10)
        
        plt.tight_layout()
        heatmap_file = os.path.join(model_output_dir, 'shap_heatmap.png')
        plt.savefig(heatmap_file, dpi=300, bbox_inches='tight')
        plt.close()
        visualization_files.append(heatmap_file)
        print(f"      ✓ SHAP热图")
    except Exception as e:
        print(f"      ❌ SHAP热图失败: {e}")
    
    # 5.5 SHAP瀑布图 (Waterfall Plot) - 多个样本
    try:
        print(f"      创建SHAP瀑布图...")
        waterfall_samples = min(10, shap_values.shape[0])
        
        for i in range(waterfall_samples):
            try:
                plt.figure(figsize=(14, 10))
                
                # 获取基准值
                if hasattr(explainer, 'expected_value'):
                    expected_value = explainer.expected_value
                else:
                    expected_value = model.predict(X_model).mean()
                
                # 获取预测值
                prediction = model.predict(X_model[i:i+1])[0]
                
                # 创建瀑布图
                shap.plots.waterfall(
                    shap.Explanation(
                        values=shap_values[i],
                        base_values=expected_value,
                        data=X_model[i],
                        feature_names=features
                    ),
                    show=False,
                    max_display=len(features)
                )
                
                plt.title(f'SHAP 瀑布图 - {model_key} - 样本 {i+1} (预测值: {prediction:.4f})', 
                         fontsize=14, fontweight='bold')
                
                plt.tight_layout()
                waterfall_file = os.path.join(model_output_dir, f'shap_waterfall_sample_{i+1}.png')
                plt.savefig(waterfall_file, dpi=300, bbox_inches='tight')
                plt.close()
                visualization_files.append(waterfall_file)
                
            except Exception as e:
                print(f"        ❌ 样本 {i+1} 瀑布图失败: {e}")
        
        print(f"      ✓ SHAP瀑布图 ({waterfall_samples}个样本)")
    except Exception as e:
        print(f"      ❌ SHAP瀑布图失败: {e}")
    
    # 5.6 SHAP决策图 (Decision Plot)
    try:
        print(f"      创建SHAP决策图...")
        
        # 限制样本数量以便可视化
        decision_samples = min(100, shap_values.shape[0])
        sample_indices = np.random.choice(shap_values.shape[0], decision_samples, replace=False)
        
        plt.figure(figsize=(14, 10))
        
        # 获取基准值
        if hasattr(explainer, 'expected_value'):
            expected_value = explainer.expected_value
        else:
            expected_value = model.predict(X_model).mean()
        
        # 创建决策图
        shap.decision_plot(
            expected_value,
            shap_values[sample_indices],
            features=features,
            show=False,
            highlight=0  # 高亮第一个样本
        )
        
        plt.title(f'SHAP 决策图 - {model_key} ({decision_samples}个样本)', fontsize=14, fontweight='bold')
        plt.xlabel('模型输出', fontsize=12)
        plt.ylabel('特征', fontsize=12)
        
        plt.tight_layout()
        decision_file = os.path.join(model_output_dir, 'shap_decision_plot.png')
        plt.savefig(decision_file, dpi=300, bbox_inches='tight')
        plt.close()
        visualization_files.append(decision_file)
        print(f"      ✓ SHAP决策图")
    except Exception as e:
        print(f"      ❌ SHAP决策图失败: {e}")
    
    return visualization_files

# ====================== 6. SHAP数据保存函数（修复版本）======================
def save_shap_data(shap_values, X_model, y_all_clean, features, model, explainer, model_output_dir, model_key):
    """
    保存完整的SHAP分析数据（修复决策图数据格式问题）
    """
    print(f"    保存完整SHAP分析数据...")
    
    data_files = []
    
    # 计算基础统计
    mean_abs_shap = np.mean(np.abs(shap_values), axis=0)
    
    # 6.1 特征重要性数据（对应条形图）
    try:
        shap_importance_df = pd.DataFrame({
            'Feature': features,
            'Mean_Absolute_SHAP_Value': mean_abs_shap,
            'Relative_Importance': mean_abs_shap / np.sum(mean_abs_shap) * 100,
            'Rank': range(1, len(features) + 1)
        })
        shap_importance_df = shap_importance_df.sort_values('Mean_Absolute_SHAP_Value', ascending=False)
        shap_importance_df['Rank'] = range(1, len(features) + 1)
        
        importance_file = os.path.join(model_output_dir, 'shap_feature_importance.xlsx')
        shap_importance_df.to_excel(importance_file, index=False)
        data_files.append(importance_file)
        print(f"      ✓ 特征重要性数据")
    except Exception as e:
        print(f"      ❌ 特征重要性数据失败: {e}")
    
    # 6.2 概要图/蜂群图数据（修改为宽格式）
    try:
        print(f"      创建概要图/蜂群图数据（宽格式）...")
        
        # 创建宽格式数据框
        summary_data_wide = {
            'Sample_Index': list(range(1, X_model.shape[0] + 1)),
            'Actual_KQ': y_all_clean,
            'Predicted_KQ': model.predict(X_model)
        }
        
        # 为每个特征添加三列：特征值、SHAP值、标准化特征值
        for feature_idx, feature_name in enumerate(features):
            # 特征值
            summary_data_wide[f'{feature_name}_Value'] = X_model[:, feature_idx]
            
            # SHAP值
            summary_data_wide[f'{feature_name}_SHAP'] = shap_values[:, feature_idx]
            
            # 标准化特征值（0-1标准化）
            feature_values = X_model[:, feature_idx]
            if feature_values.max() != feature_values.min():
                normalized_values = (feature_values - feature_values.min()) / (feature_values.max() - feature_values.min())
            else:
                normalized_values = np.full_like(feature_values, 0.5)
            summary_data_wide[f'{feature_name}_Normalized'] = normalized_values
        
        summary_df_wide = pd.DataFrame(summary_data_wide)
        summary_file_wide = os.path.join(model_output_dir, 'shap_summary_beeswarm_data.xlsx')
        summary_df_wide.to_excel(summary_file_wide, index=False)
        data_files.append(summary_file_wide)
        print(f"      ✓ 概要图/蜂群图数据（宽格式，便于Origin处理）")
        
        # 同时保存长格式版本，以备需要
        summary_data_long = []
        for sample_idx in range(X_model.shape[0]):
            for feature_idx, feature_name in enumerate(features):
                summary_data_long.append({
                    'Sample_Index': sample_idx + 1,
                    'Feature_Name': feature_name,
                    'Feature_Value': X_model[sample_idx, feature_idx],
                    'SHAP_Value': shap_values[sample_idx, feature_idx],
                    'Feature_Value_Normalized': (X_model[sample_idx, feature_idx] - X_model[:, feature_idx].min()) / 
                                              (X_model[:, feature_idx].max() - X_model[:, feature_idx].min()) if X_model[:, feature_idx].max() != X_model[:, feature_idx].min() else 0.5,
                    'Actual_KQ': y_all_clean[sample_idx],
                    'Predicted_KQ': model.predict(X_model[sample_idx:sample_idx+1])[0]
                })
        
        summary_df_long = pd.DataFrame(summary_data_long)
        summary_file_long = os.path.join(model_output_dir, 'shap_summary_beeswarm_data_long_format.xlsx')
        summary_df_long.to_excel(summary_file_long, index=False)
        data_files.append(summary_file_long)
        print(f"      ✓ 概要图/蜂群图数据（长格式备份）")
        
    except Exception as e:
        print(f"      ❌ 概要图/蜂群图数据失败: {e}")
    
    # 6.3 热图数据
    try:
        importance_order = np.argsort(mean_abs_shap)[::-1]
        ordered_feature_names = [features[i] for i in importance_order]
        ordered_shap_values = shap_values[:, importance_order]
        
        heatmap_df = pd.DataFrame(ordered_shap_values, columns=ordered_feature_names)
        heatmap_df.insert(0, 'Sample_Index', range(1, len(X_model) + 1))
        heatmap_df.insert(1, 'Actual_KQ', y_all_clean)
        heatmap_df.insert(2, 'Predicted_KQ', model.predict(X_model))
        
        # 添加特征值（按重要性排序）
        ordered_feature_values = X_model[:, importance_order]
        for i, feature_name in enumerate(ordered_feature_names):
            heatmap_df[f'{feature_name}_Value'] = ordered_feature_values[:, i]
        
        heatmap_file = os.path.join(model_output_dir, 'shap_heatmap_data.xlsx')
        heatmap_df.to_excel(heatmap_file, index=False)
        data_files.append(heatmap_file)
        print(f"      ✓ 热图数据")
    except Exception as e:
        print(f"      ❌ 热图数据失败: {e}")
    
    # 6.4 瀑布图数据（前20个样本）
    try:
        waterfall_data = []
        
        # 获取基准值 - 🔧 修复：确保是标量
        if hasattr(explainer, 'expected_value'):
            expected_value = explainer.expected_value
            if isinstance(expected_value, (list, np.ndarray)):
                expected_value = float(expected_value[0])
            else:
                expected_value = float(expected_value)
        else:
            expected_value = float(model.predict(X_model).mean())
        
        print(f"      使用基准值: {expected_value}")
        
        # 保存前20个样本的瀑布图数据
        waterfall_samples = min(20, X_model.shape[0])
        for i in range(waterfall_samples):
            prediction = model.predict(X_model[i:i+1])[0]
            
            # 基础数据
            sample_data = {
                'Sample_Index': i + 1,
                'Base_Value': expected_value,
                'Prediction': prediction,
                'Actual_KQ': y_all_clean[i],
                'Total_SHAP_Effect': np.sum(shap_values[i]),
                'Prediction_Error': prediction - y_all_clean[i],
                'Absolute_Prediction_Error': abs(prediction - y_all_clean[i])
            }
            
            # 每个特征的SHAP值和特征值
            for j, feature in enumerate(features):
                sample_data[f'SHAP_{feature}'] = shap_values[i, j]
                sample_data[f'Value_{feature}'] = X_model[i, j]
            
            # 按SHAP绝对值排序的特征贡献
            abs_shap = np.abs(shap_values[i])
            sorted_indices = np.argsort(abs_shap)[::-1]
            
            cumulative_effect = expected_value
            for rank, j in enumerate(sorted_indices):
                feature = features[j]
                feature_effect = shap_values[i, j]
                cumulative_effect += feature_effect
                
                sample_data[f'Rank_{rank+1}_Feature'] = feature
                sample_data[f'Rank_{rank+1}_SHAP'] = feature_effect
                sample_data[f'Rank_{rank+1}_Value'] = X_model[i, j]
                sample_data[f'Rank_{rank+1}_Abs_SHAP'] = abs(feature_effect)
                sample_data[f'Cumulative_After_Rank_{rank+1}'] = cumulative_effect
            
            waterfall_data.append(sample_data)
        
        waterfall_df = pd.DataFrame(waterfall_data)
        waterfall_file = os.path.join(model_output_dir, 'shap_waterfall_data.xlsx')
        waterfall_df.to_excel(waterfall_file, index=False)
        data_files.append(waterfall_file)
        print(f"      ✓ 瀑布图数据 (前{waterfall_samples}个样本)")
    except Exception as e:
        print(f"      ❌ 瀑布图数据失败: {e}")
    
    # 6.5 瀑布图数据（所有样本）
    try:
        print(f"      正在保存所有 {X_model.shape[0]} 个样本的瀑布图数据...")
        
        waterfall_data_all = []
        
        # 获取基准值 - 🔧 修复：确保是标量
        if hasattr(explainer, 'expected_value'):
            expected_value = explainer.expected_value
            if isinstance(expected_value, (list, np.ndarray)):
                expected_value = float(expected_value[0])
            else:
                expected_value = float(expected_value)
        else:
            expected_value = float(model.predict(X_model).mean())
        
        for i in range(X_model.shape[0]):
            prediction = model.predict(X_model[i:i+1])[0]
            
            # 基础数据
            sample_data = {
                'Sample_Index': i + 1,
                'Base_Value': expected_value,
                'Prediction': prediction,
                'Actual_KQ': y_all_clean[i],
                'Total_SHAP_Effect': np.sum(shap_values[i]),
                'Prediction_Error': prediction - y_all_clean[i],
                'Absolute_Prediction_Error': abs(prediction - y_all_clean[i])
            }
            
            # 每个特征的SHAP值和特征值
            for j, feature in enumerate(features):
                sample_data[f'SHAP_{feature}'] = shap_values[i, j]
                sample_data[f'Value_{feature}'] = X_model[i, j]
            
            # 按SHAP绝对值排序的特征贡献
            abs_shap = np.abs(shap_values[i])
            sorted_indices = np.argsort(abs_shap)[::-1]
            
            cumulative_effect = expected_value
            for rank, j in enumerate(sorted_indices):
                feature = features[j]
                feature_effect = shap_values[i, j]
                cumulative_effect += feature_effect
                
                sample_data[f'Rank_{rank+1}_Feature'] = feature
                sample_data[f'Rank_{rank+1}_SHAP'] = feature_effect
                sample_data[f'Rank_{rank+1}_Value'] = X_model[i, j]
                sample_data[f'Rank_{rank+1}_Abs_SHAP'] = abs(feature_effect)
                sample_data[f'Cumulative_After_Rank_{rank+1}'] = cumulative_effect
            
            waterfall_data_all.append(sample_data)
        
        waterfall_df_all = pd.DataFrame(waterfall_data_all)
        waterfall_file_all = os.path.join(model_output_dir, 'shap_waterfall_data_all_samples.xlsx')
        waterfall_df_all.to_excel(waterfall_file_all, index=False)
        data_files.append(waterfall_file_all)
        print(f"      ✓ 瀑布图数据 (所有 {X_model.shape[0]} 个样本)")
    except Exception as e:
        print(f"      ❌ 瀑布图数据（所有样本）失败: {e}")
    
    # 6.6 决策图数据（随机100个样本）- 🔧 修复版本
    try:
        decision_samples = min(100, X_model.shape[0])
        sample_indices_decision = np.random.choice(X_model.shape[0], decision_samples, replace=False)
        
        # 获取基准值 - 🔧 修复：确保是标量
        if hasattr(explainer, 'expected_value'):
            expected_value = explainer.expected_value
            if isinstance(expected_value, (list, np.ndarray)):
                expected_value = float(expected_value[0])
            else:
                expected_value = float(expected_value)
        else:
            expected_value = float(model.predict(X_model).mean())
        
        print(f"      决策图基准值（随机样本）: {expected_value}")
        
        decision_data = []
        for i in sample_indices_decision:
            sample_data = {
                'Sample_Index': i + 1,
                'Base_Value': expected_value,
                'Prediction': model.predict(X_model[i:i+1])[0],
                'Actual_KQ': y_all_clean[i],
                'Prediction_Error': model.predict(X_model[i:i+1])[0] - y_all_clean[i],
                'Absolute_Prediction_Error': abs(model.predict(X_model[i:i+1])[0] - y_all_clean[i])
            }
            
            # 🔧 修复：累积SHAP效应（按特征重要性顺序）
            cumulative_shap = expected_value  # 确保是标量
            sample_data['Step_0_Cumulative'] = cumulative_shap
            sample_data['Step_0_Feature'] = 'Base_Value'
            
            importance_order = np.argsort(mean_abs_shap)[::-1]
            for step, j in enumerate(importance_order):
                feature = features[j]
                shap_contribution = float(shap_values[i, j])  # 🔧 确保是标量
                cumulative_shap += shap_contribution
                
                sample_data[f'Step_{step+1}_Feature'] = feature
                sample_data[f'Step_{step+1}_SHAP'] = shap_contribution
                sample_data[f'Step_{step+1}_Value'] = float(X_model[i, j])
                sample_data[f'Step_{step+1}_Cumulative'] = cumulative_shap  # 🔧 这里应该是正确的累积值
                sample_data[f'Step_{step+1}_SHAP_Abs'] = abs(shap_contribution)
            
            # 添加最终验证
            sample_data['Final_Cumulative'] = cumulative_shap
            sample_data['Cumulative_Equals_Prediction'] = abs(cumulative_shap - sample_data['Prediction']) < 1e-6
            
            decision_data.append(sample_data)
        
        # 🔧 添加验证输出（前5个样本）
        print(f"      验证决策图数据计算（前5个样本）:")
        for idx in range(min(5, len(decision_data))):
            sample = decision_data[idx]
            step_1_calculated = sample['Step_0_Cumulative'] + sample['Step_1_SHAP']
            print(f"        样本 {sample['Sample_Index']}: Base={sample['Base_Value']:.6f}, "
                  f"Step1_计算={step_1_calculated:.6f}, Step1_实际={sample['Step_1_Cumulative']:.6f}, "
                  f"匹配={abs(step_1_calculated - sample['Step_1_Cumulative']) < 1e-6}")
        
        decision_df = pd.DataFrame(decision_data)
        decision_file = os.path.join(model_output_dir, 'shap_decision_plot_data.xlsx')
        decision_df.to_excel(decision_file, index=False)
        data_files.append(decision_file)
        print(f"      ✓ 决策图数据 (随机{decision_samples}个样本)")
    except Exception as e:
        print(f"      ❌ 决策图数据失败: {e}")
    
    # 6.7 决策图数据（所有样本）- 🔧 修复版本
    try:
        print(f"      正在保存所有 {X_model.shape[0]} 个样本的决策图数据...")
        
        # 获取基准值 - 🔧 修复：确保是标量
        if hasattr(explainer, 'expected_value'):
            expected_value = explainer.expected_value
            if isinstance(expected_value, (list, np.ndarray)):
                expected_value = float(expected_value[0])
            else:
                expected_value = float(expected_value)
        else:
            expected_value = float(model.predict(X_model).mean())
        
        print(f"      决策图基准值（所有样本）: {expected_value}")
        
        decision_data_all = []
        for i in range(X_model.shape[0]):
            sample_data = {
                'Sample_Index': i + 1,
                'Base_Value': expected_value,
                'Prediction': model.predict(X_model[i:i+1])[0],
                'Actual_KQ': y_all_clean[i],
                'Prediction_Error': model.predict(X_model[i:i+1])[0] - y_all_clean[i],
                'Absolute_Prediction_Error': abs(model.predict(X_model[i:i+1])[0] - y_all_clean[i])
            }
            
            # 🔧 修复：累积SHAP效应（按特征重要性顺序）
            cumulative_shap = expected_value  # 确保是标量
            sample_data['Step_0_Cumulative'] = cumulative_shap
            sample_data['Step_0_Feature'] = 'Base_Value'
            
            importance_order = np.argsort(mean_abs_shap)[::-1]
            for step, j in enumerate(importance_order):
                feature = features[j]
                shap_contribution = float(shap_values[i, j])  # 🔧 确保是标量
                cumulative_shap += shap_contribution
                
                sample_data[f'Step_{step+1}_Feature'] = feature
                sample_data[f'Step_{step+1}_SHAP'] = shap_contribution
                sample_data[f'Step_{step+1}_Value'] = float(X_model[i, j])
                sample_data[f'Step_{step+1}_Cumulative'] = cumulative_shap  # 🔧 这里应该是正确的累积值
                sample_data[f'Step_{step+1}_SHAP_Abs'] = abs(shap_contribution)
            
            # 添加最终验证
            sample_data['Final_Cumulative'] = cumulative_shap
            sample_data['Cumulative_Equals_Prediction'] = abs(cumulative_shap - sample_data['Prediction']) < 1e-6
            
            decision_data_all.append(sample_data)
        
        # 🔧 添加验证输出（前5个样本）
        print(f"      验证决策图数据计算（所有样本，前5个样本）:")
        for idx in range(min(5, len(decision_data_all))):
            sample = decision_data_all[idx]
            step_1_calculated = sample['Step_0_Cumulative'] + sample['Step_1_SHAP']
            print(f"        样本 {sample['Sample_Index']}: Base={sample['Base_Value']:.6f}, "
                  f"Step1_计算={step_1_calculated:.6f}, Step1_实际={sample['Step_1_Cumulative']:.6f}, "
                  f"匹配={abs(step_1_calculated - sample['Step_1_Cumulative']) < 1e-6}")
        
        decision_df_all = pd.DataFrame(decision_data_all)
        decision_file_all = os.path.join(model_output_dir, 'shap_decision_plot_data_all_samples.xlsx')
        decision_df_all.to_excel(decision_file_all, index=False)
        data_files.append(decision_file_all)
        print(f"      ✓ 决策图数据 (所有 {X_model.shape[0]} 个样本)")
    except Exception as e:
        print(f"      ❌ 决策图数据（所有样本）失败: {e}")
    
    # 6.8 每个样本的完整SHAP值
    try:
        # 原始特征值
        sample_data_df = pd.DataFrame(X_model, columns=features)
        sample_data_df.insert(0, 'Sample_Index', range(1, len(X_model) + 1))
        sample_data_df.insert(1, 'Actual_KQ', y_all_clean)
        sample_data_df.insert(2, 'Predicted_KQ', model.predict(X_model))
        
        # SHAP值
        shap_values_df = pd.DataFrame(shap_values, columns=[f'SHAP_{col}' for col in features])
        
        # 计算每个样本的SHAP统计
        shap_stats_df = pd.DataFrame({
            'Total_SHAP_Effect': np.sum(shap_values, axis=1),
            'Total_Absolute_SHAP': np.sum(np.abs(shap_values), axis=1),
            'Max_Positive_SHAP': np.max(np.where(shap_values > 0, shap_values, 0), axis=1),
            'Max_Negative_SHAP': np.min(np.where(shap_values < 0, shap_values, 0), axis=1),
            'SHAP_Range': np.max(shap_values, axis=1) - np.min(shap_values, axis=1)
        })
        
        # 合并所有数据
        combined_df = pd.concat([sample_data_df, shap_values_df, shap_stats_df], axis=1)
        complete_file = os.path.join(model_output_dir, 'shap_values_all_samples.xlsx')
        combined_df.to_excel(complete_file, index=False)
        data_files.append(complete_file)
        print(f"      ✓ 完整样本SHAP值数据")
    except Exception as e:
        print(f"      ❌ 完整样本SHAP值数据失败: {e}")
    
    # 6.9 SHAP统计分析
    try:
        stats_data = []
        for i, feature in enumerate(features):
            feature_shap = shap_values[:, i]
            feature_values = X_model[:, i]
            
            # 计算分位数
            shap_percentiles = np.percentile(feature_shap, [10, 25, 50, 75, 90])
            value_percentiles = np.percentile(feature_values, [10, 25, 50, 75, 90])
            
            stats_data.append({
                'Feature': feature,
                'SHAP_Mean': np.mean(feature_shap),
                'SHAP_Std': np.std(feature_shap),
                'SHAP_Min': np.min(feature_shap),
                'SHAP_Max': np.max(feature_shap),
                'SHAP_10th_Percentile': shap_percentiles[0],
                'SHAP_25th_Percentile': shap_percentiles[1],
                'SHAP_Median': shap_percentiles[2],
                'SHAP_75th_Percentile': shap_percentiles[3],
                'SHAP_90th_Percentile': shap_percentiles[4],
                'SHAP_Mean_Abs': np.mean(np.abs(feature_shap)),
                'Feature_Mean': np.mean(feature_values),
                'Feature_Std': np.std(feature_values),
                'Feature_Min': np.min(feature_values),
                'Feature_Max': np.max(feature_values),
                'Feature_10th_Percentile': value_percentiles[0],
                'Feature_25th_Percentile': value_percentiles[1],
                'Feature_Median': value_percentiles[2],
                'Feature_75th_Percentile': value_percentiles[3],
                'Feature_90th_Percentile': value_percentiles[4],
                'SHAP_Feature_Correlation': np.corrcoef(feature_values, feature_shap)[0, 1],
                'Relative_Importance': mean_abs_shap[i] / np.sum(mean_abs_shap) * 100,
                'Positive_SHAP_Count': np.sum(feature_shap > 0),
                'Negative_SHAP_Count': np.sum(feature_shap < 0),
                'Zero_SHAP_Count': np.sum(feature_shap == 0),
                'Positive_SHAP_Percentage': np.sum(feature_shap > 0) / len(feature_shap) * 100,
                'Negative_SHAP_Percentage': np.sum(feature_shap < 0) / len(feature_shap) * 100
            })
        
        stats_df = pd.DataFrame(stats_data)
        stats_df = stats_df.sort_values('SHAP_Mean_Abs', ascending=False)
        stats_file = os.path.join(model_output_dir, 'shap_statistical_analysis.xlsx')
        stats_df.to_excel(stats_file, index=False)
        data_files.append(stats_file)
        print(f"      ✓ SHAP统计分析数据")
    except Exception as e:
        print(f"      ❌ SHAP统计分析数据失败: {e}")
    
    # 6.10 数据文件索引（更新）
    try:
        file_index = {
            'File_Name': [
                'shap_feature_importance.xlsx',
                'shap_summary_beeswarm_data.xlsx',
                'shap_summary_beeswarm_data_long_format.xlsx',
                'shap_heatmap_data.xlsx',
                'shap_waterfall_data.xlsx',
                'shap_waterfall_data_all_samples.xlsx',
                'shap_decision_plot_data.xlsx',
                'shap_decision_plot_data_all_samples.xlsx',
                'shap_values_all_samples.xlsx',
                'shap_statistical_analysis.xlsx'
            ],
            'Corresponding_Visualization': [
                'SHAP条形图 (Bar Plot)',
                'SHAP概要图 & 蜂群图 (Summary Plot & Beeswarm Plot)',
                'SHAP概要图 & 蜂群图 (Summary Plot & Beeswarm Plot)',
                'SHAP热图 (Heatmap)',
                'SHAP瀑布图 (Waterfall Plot) - 前20个样本',
                'SHAP瀑布图 (Waterfall Plot) - 所有样本',
                'SHAP决策图 (Decision Plot) - 随机100个样本',
                'SHAP决策图 (Decision Plot) - 所有样本',
                '所有样本完整数据 (All Samples Complete Data)',
                'SHAP统计分析 (Statistical Analysis)'
            ],
            'Description': [
                '特征重要性排序，平均绝对SHAP值',
                '每个样本每个特征的SHAP值和特征值，用于创建散点图（宽格式，推荐用于Origin）',
                '每个样本每个特征的SHAP值和特征值，用于创建散点图（长格式，备份）',
                '按特征重要性排序的SHAP值矩阵，包含样本和特征信息',
                '瀑布图所需的累积SHAP效应数据，包含前20个样本',
                '瀑布图所需的累积SHAP效应数据，包含所有样本（Origin推荐）',
                '决策路径数据，显示SHAP值的累积过程，包含随机100个样本（已修复累积值计算）',
                '决策路径数据，显示SHAP值的累积过程，包含所有样本（Origin推荐，已修复累积值计算）',
                '所有样本的完整特征值、SHAP值和预测信息',
                '详细的SHAP值统计分析，包含分位数、相关性等指标'
            ],
            'Sample_Count': [
                f'{len(features)}个特征',
                f'{X_model.shape[0]}个样本（行）× {len(features)*3+3}列（每特征3列+基础3列）',
                f'{X_model.shape[0]}个样本 × {len(features)}个特征',
                f'{X_model.shape[0]}个样本 × {len(features)}个特征',
                '20个样本',
                f'{X_model.shape[0]}个样本',
                '100个样本',
                f'{X_model.shape[0]}个样本',
                f'{X_model.shape[0]}个样本',
                f'{len(features)}个特征的统计指标'
            ],
            'Origin_Recommended': [
                '是',
                '是（主要推荐，宽格式）', 
                '否（备份，长格式）',
                '是',
                '否（样本有限）',
                '是（推荐使用）',
                '否（样本有限）',
                '是（推荐使用，已修复）',
                '是',
                '是'
            ]
        }
        
        index_df = pd.DataFrame(file_index)
        index_file = os.path.join(model_output_dir, 'data_files_index.xlsx')
        index_df.to_excel(index_file, index=False)
        data_files.append(index_file)
        print(f"      ✓ 数据文件索引")
    except Exception as e:
        print(f"      ❌ 数据文件索引失败: {e}")
    
    return data_files

# ====================== 7. 单个模型SHAP分析函数 ======================
def analyze_single_model_shap(model_info, model_index, total_models, all_features_name, df, data_info, scaler, shap_output_dir):
    """
    对单个模型进行完整的SHAP分析
    """
    print(f"\n[{model_index+1}/{total_models}] 分析模型: {model_info['model_key']}")
    print(f"  方法: {model_info['method']}, 模型: {model_info['model']}, K={model_info['k']}")
    print(f"  Test R²: {model_info['test_r2']:.4f}, Test MAE: {model_info['test_mae']:.4f}")
    
    start_time = time.time()
    
    # 为每个模型创建单独的输出目录
    model_safe_name = model_info['model_key'].replace('/', '_').replace('\\', '_').replace(':', '_')
    model_output_dir = os.path.join(shap_output_dir, f"Model_{model_index+1:02d}_{model_safe_name}")
    if not os.path.exists(model_output_dir):
        os.makedirs(model_output_dir)
    
    try:
        # 1. 准备数据
        X_model, y_all_clean, model, features, final_data_type = prepare_shap_data_for_model(
            model_info, all_features_name, df, data_info, scaler
        )
        
        # 2. 创建SHAP解释器并计算SHAP值
        print(f"  创建SHAP解释器...")
        try:
            # 优先使用TreeExplainer
            explainer = shap.TreeExplainer(model)
            shap_values = explainer.shap_values(X_model)
            print(f"    ✓ TreeExplainer成功")
        except Exception as e:
            print(f"    TreeExplainer失败，尝试Explainer: {e}")
            try:
                explainer = shap.Explainer(model)
                shap_values = explainer(X_model)
                if hasattr(shap_values, 'values'):
                    shap_values = shap_values.values
                print(f"    ✓ Explainer成功")
            except Exception as e2:
                print(f"    Explainer失败，使用KernelExplainer: {e2}")
                bg_samples = min(100, X_model.shape[0])
                bg_data = X_model[:bg_samples]
                explainer = shap.KernelExplainer(model.predict, bg_data)
                shap_values = explainer.shap_values(X_model)
                print(f"    ✓ KernelExplainer成功")
        
        # 确保SHAP值是正确的格式
        if len(shap_values.shape) > 2:
            shap_values = np.squeeze(shap_values)
        
        print(f"    SHAP值形状: {shap_values.shape}")
        
        # 3. 创建完整可视化（包含所有6种类型）
        print(f"  创建完整可视化...")
        visualization_files = create_shap_visualizations(
            shap_values, X_model, features, model_output_dir, model_info['model_key'], 
            explainer, model, y_all_clean
        )
        
        # 4. 保存完整数据（包含修复后的决策图数据）
        print(f"  保存完整数据...")
        data_files = save_shap_data(
            shap_values, X_model, y_all_clean, features, model, explainer, model_output_dir, model_info['model_key']
        )
        
        # 5. 保存模型信息
        model_info_data = {
            'Model_Key': model_info['model_key'],
            'Method': model_info['method'],
            'ML_Model': model_info['model'],
            'K': model_info['k'],
            'Train_R2': model_info['train_r2'],
            'Test_R2': model_info['test_r2'],
            'Train_MAE': model_info['train_mae'],
            'Test_MAE': model_info['test_mae'],
            'Train_RMSLE': model_info['train_rmsle'],
            'Test_RMSLE': model_info['test_rmsle'],
            'Is_Tree_Based': model_info['is_tree_based'],
            'Is_Advanced_Method': model_info['is_advanced'],
            'Is_Focus_Combination': model_info['is_focus_combination'],
            'Data_Type': final_data_type,
            'Total_Samples': X_model.shape[0],
            'Features': features,
            'Selected_Indices': model_info['sel_idx']
        }
        
        # 保存模型信息到文本文件
        with open(os.path.join(model_output_dir, 'model_info.txt'), 'w', encoding='utf-8') as f:
            f.write("=== 模型SHAP分析信息（修复版）===\n\n")
            for key, value in model_info_data.items():
                if key == 'Features':
                    f.write(f"{key}:\n")
                    for i, feature in enumerate(value, 1):
                        f.write(f"  {i}. {feature}\n")
                elif key == 'Selected_Indices':
                    f.write(f"{key}: {value}\n")
                else:
                    f.write(f"{key}: {value}\n")
            
            f.write(f"\n=== SHAP分析结果文件 ===\n")
            f.write(f"可视化文件:\n")
            f.write(f"  1. shap_summary_plot.png - SHAP概要图\n")
            f.write(f"  2. shap_beeswarm_plot.png - SHAP蜂群图\n")
            f.write(f"  3. shap_bar_plot.png - SHAP条形图\n")
            f.write(f"  4. shap_heatmap.png - SHAP热图\n")
            f.write(f"  5. shap_waterfall_sample_*.png - SHAP瀑布图 (多个样本)\n")
            f.write(f"  6. shap_decision_plot.png - SHAP决策图\n")
            f.write(f"\n数据文件:\n")
            f.write(f"  1. shap_feature_importance.xlsx - 特征重要性数据\n")
            f.write(f"  2. shap_summary_beeswarm_data.xlsx - 概要图/蜂群图数据（宽格式，推荐用于Origin）\n")
            f.write(f"  3. shap_summary_beeswarm_data_long_format.xlsx - 概要图/蜂群图数据（长格式备份）\n")
            f.write(f"  4. shap_heatmap_data.xlsx - 热图数据\n")
            f.write(f"  5. shap_waterfall_data.xlsx - 瀑布图数据 (前20个样本)\n")
            f.write(f"  6. shap_waterfall_data_all_samples.xlsx - 瀑布图数据 (所有样本)\n")
            f.write(f"  7. shap_decision_plot_data.xlsx - 决策图数据 (随机100个样本)\n")
            f.write(f"  8. shap_decision_plot_data_all_samples.xlsx - 决策图数据 (所有样本)【已修复】\n")
            f.write(f"  9. shap_values_all_samples.xlsx - 完整样本SHAP值\n")
            f.write(f"  10. shap_statistical_analysis.xlsx - SHAP统计分析\n")
            f.write(f"  11. data_files_index.xlsx - 数据文件索引\n")
            f.write(f"\n=== 🔧 主要修复 ===\n")
            f.write(f"1. 修复了决策图数据中累积值计算错误的问题\n")
            f.write(f"   - Step_0_Cumulative = Base_Value (正确)\n")
            f.write(f"   - Step_1_Cumulative = Base_Value + Step_1_SHAP (修复后)\n")
            f.write(f"   - Step_N_Cumulative = 前一步累积值 + Step_N_SHAP (修复后)\n")
            f.write(f"2. 确保基准值(expected_value)是标量而非数组格式\n")
            f.write(f"3. 添加了累积值计算的验证机制\n")
            f.write(f"4. 保持宽格式概要图数据便于Origin使用\n")
        
        elapsed_time = time.time() - start_time
        print(f"  ✓ 模型 {model_info['model_key']} 分析完成 (耗时: {elapsed_time:.1f}秒)")
        
        return {
            'model_key': model_info['model_key'],
            'output_dir': model_output_dir,
            'visualization_files': visualization_files,
            'data_files': data_files,
            'model_info': model_info_data,
            'success': True,
            'error': None,
            'elapsed_time': elapsed_time
        }
        
    except Exception as e:
        elapsed_time = time.time() - start_time
        print(f"  ❌ 模型 {model_info['model_key']} 分析失败: {e}")
        
        return {
            'model_key': model_info['model_key'],
            'output_dir': model_output_dir,
            'visualization_files': [],
            'data_files': [],
            'model_info': None,
            'success': False,
            'error': str(e),
            'elapsed_time': elapsed_time
        }

# ====================== 8. 主分析流程 ======================
print(f"\n=== 开始多模型SHAP分析（修复版）===")
print(f"总计模型数量: {len(best_models)}")

all_results = []
successful_analyses = 0
failed_analyses = 0

# 分析每个模型
for i, model_info in enumerate(best_models):
    result = analyze_single_model_shap(
        model_info, i, len(best_models), all_features_name, df, data_info, scaler, shap_output_dir
    )
    all_results.append(result)
    
    if result['success']:
        successful_analyses += 1
    else:
        failed_analyses += 1

# ====================== 9. 创建总体分析报告 ======================
print(f"\n=== 创建总体分析报告 ===")

# 9.1 创建总体结果汇总
summary_data = []
for result in all_results:
    if result['success']:
        model_info = result['model_info']
        summary_data.append({
            'Model_Index': len(summary_data) + 1,
            'Model_Key': result['model_key'],
            'Method': model_info['Method'],
            'ML_Model': model_info['ML_Model'],
            'K': model_info['K'],
            'Test_R2': model_info['Test_R2'],
            'Test_MAE': model_info['Test_MAE'],
            'Is_Advanced_Method': model_info['Is_Advanced_Method'],
            'Is_Focus_Combination': model_info['Is_Focus_Combination'],
            'Output_Directory': os.path.basename(result['output_dir']),
            'Visualization_Count': len(result['visualization_files']),
            'Data_File_Count': len(result['data_files']),
            'Analysis_Time_Seconds': result['elapsed_time'],
            'Features': ', '.join(model_info['Features']),
            'Total_Samples': model_info['Total_Samples']
        })
    else:
        summary_data.append({
            'Model_Index': len(summary_data) + 1,
            'Model_Key': result['model_key'],
            'Method': 'N/A',
            'ML_Model': 'N/A',
            'K': 'N/A',
            'Test_R2': 'N/A',
            'Test_MAE': 'N/A',
            'Is_Advanced_Method': 'N/A',
            'Is_Focus_Combination': 'N/A',
            'Output_Directory': 'FAILED',
            'Visualization_Count': 0,
            'Data_File_Count': 0,
            'Analysis_Time_Seconds': result['elapsed_time'],
            'Features': f'FAILED: {result["error"]}',
            'Total_Samples': 'N/A'
        })

summary_df = pd.DataFrame(summary_data)
summary_file = os.path.join(shap_output_dir, 'SHAP_Analysis_Summary_Fixed.xlsx')
summary_df.to_excel(summary_file, index=False)

# 9.2 创建目录索引
directory_index = []
for result in all_results:
    if result['success']:
        directory_index.append({
            'Model_Key': result['model_key'],
            'Directory_Name': os.path.basename(result['output_dir']),
            'Full_Path': result['output_dir'],
            'Visualization_Count': len(result['visualization_files']),
            'Data_File_Count': len(result['data_files']),
            'Status': 'SUCCESS'
        })
    else:
        directory_index.append({
            'Model_Key': result['model_key'],
            'Directory_Name': 'FAILED',
            'Full_Path': result['output_dir'],
            'Visualization_Count': 0,
            'Data_File_Count': 0,
            'Status': f'FAILED: {result["error"]}'
        })

directory_df = pd.DataFrame(directory_index)
directory_file = os.path.join(shap_output_dir, 'Directory_Index_Fixed.xlsx')
directory_df.to_excel(directory_file, index=False)

# 9.3 创建总体报告文件
total_time = sum(result['elapsed_time'] for result in all_results)
with open(os.path.join(shap_output_dir, 'Multi_Model_SHAP_Analysis_Report_Fixed.txt'), 'w', encoding='utf-8') as f:
    f.write("=== 多模型SHAP分析总体报告（修复版）===\n\n")
    f.write(f"分析时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write(f"总分析时间: {total_time:.1f} 秒 ({total_time/60:.1f} 分钟)\n\n")
    
    f.write(f"=== 🔧 主要修复 ===\n")
    f.write(f"✓ 决策图数据累积值计算错误修复\n")
    f.write(f"  - 问题：Step_0_Cumulative 和 Step_1_Cumulative 显示相同值\n")
    f.write(f"  - 原因：expected_value 数组格式导致累积计算错误\n")
    f.write(f"  - 修复：确保 expected_value 为标量，正确计算累积SHAP值\n")
    f.write(f"  - 验证：添加累积值计算验证，确保数据正确性\n\n")
    
    f.write(f"✓ 数据格式优化（保持之前改进）\n")
    f.write(f"  - shap_summary_beeswarm_data.xlsx 使用宽格式\n")
    f.write(f"  - 每行代表一个样本（而非每行一个特征-样本组合）\n")
    f.write(f"  - 每个特征有3列：特征名_Value, 特征名_SHAP, 特征名_Normalized\n")
    f.write(f"  - 更便于在Origin等软件中进行数据处理和可视化\n\n")
    
    f.write(f"=== 分析统计 ===\n")
    f.write(f"总模型数量: {len(best_models)}\n")
    f.write(f"成功分析: {successful_analyses}\n")
    f.write(f"失败分析: {failed_analyses}\n")
    f.write(f"成功率: {successful_analyses/len(best_models)*100:.1f}%\n\n")
    
    f.write(f"=== 成功分析的模型 ===\n")
    for i, result in enumerate(all_results):
        if result['success']:
            model_info = result['model_info']
            f.write(f"{i+1}. {result['model_key']}\n")
            f.write(f"   方法: {model_info['Method']}, 模型: {model_info['ML_Model']}, K={model_info['K']}\n")
            f.write(f"   Test R²: {model_info['Test_R2']:.4f}, Test MAE: {model_info['Test_MAE']:.4f}\n")
            f.write(f"   输出目录: {os.path.basename(result['output_dir'])}\n")
            f.write(f"   可视化文件: {len(result['visualization_files'])}个\n")
            f.write(f"   数据文件: {len(result['data_files'])}个\n")
            f.write(f"   分析时间: {result['elapsed_time']:.1f}秒\n\n")
    
    if failed_analyses > 0:
        f.write(f"=== 失败分析的模型 ===\n")
        for i, result in enumerate(all_results):
            if not result['success']:
                f.write(f"{i+1}. {result['model_key']}\n")
                f.write(f"   错误: {result['error']}\n")
                f.write(f"   尝试时间: {result['elapsed_time']:.1f}秒\n\n")
    
    f.write(f"=== 输出文件说明 ===\n")
    f.write(f"1. SHAP_Analysis_Summary_Fixed.xlsx - 所有模型的分析结果汇总\n")
    f.write(f"2. Directory_Index_Fixed.xlsx - 输出目录索引\n")
    f.write(f"3. Multi_Model_SHAP_Analysis_Report_Fixed.txt - 本报告文件\n")
    f.write(f"4. Model_XX_[模型名]/ - 每个模型的详细SHAP分析结果\n\n")
    
    f.write(f"=== 每个模型目录包含的文件（修复版）===\n")
    f.write(f"可视化文件 (6种类型):\n")
    f.write(f"  - shap_summary_plot.png (SHAP概要图)\n")
    f.write(f"  - shap_beeswarm_plot.png (SHAP蜂群图)\n")
    f.write(f"  - shap_bar_plot.png (SHAP条形图)\n")
    f.write(f"  - shap_heatmap.png (SHAP热图)\n")
    f.write(f"  - shap_waterfall_sample_*.png (SHAP瀑布图，多个样本)\n")
    f.write(f"  - shap_decision_plot.png (SHAP决策图)\n\n")
    f.write(f"数据文件 (11个文件):\n")
    f.write(f"  - shap_feature_importance.xlsx (特征重要性数据)\n")
    f.write(f"  - shap_summary_beeswarm_data.xlsx (概要图/蜂群图数据-宽格式【推荐用于Origin】)\n")
    f.write(f"  - shap_summary_beeswarm_data_long_format.xlsx (概要图/蜂群图数据-长格式备份)\n")
    f.write(f"  - shap_heatmap_data.xlsx (热图数据)\n")
    f.write(f"  - shap_waterfall_data.xlsx (瀑布图数据-前20个样本)\n")
    f.write(f"  - shap_waterfall_data_all_samples.xlsx (瀑布图数据-所有样本)\n")
    f.write(f"  - shap_decision_plot_data.xlsx (决策图数据-随机100个样本)\n")
    f.write(f"  - shap_decision_plot_data_all_samples.xlsx (决策图数据-所有样本【已修复累积值计算】)\n")
    f.write(f"  - shap_values_all_samples.xlsx (完整SHAP值数据)\n")
    f.write(f"  - shap_statistical_analysis.xlsx (统计分析数据)\n")
    f.write(f"  - data_files_index.xlsx (数据文件索引)\n")
    f.write(f"  - model_info.txt (模型信息)\n\n")

# ====================== 10. 输出最终结果 ======================
print(f"\n=== 多模型SHAP分析完成（修复版）===")
print(f"🎉 总体结果:")
print(f"  📊 总模型数量: {len(best_models)}")
print(f"  ✅ 成功分析: {successful_analyses}")
print(f"  ❌ 失败分析: {failed_analyses}")
print(f"  📈 成功率: {successful_analyses/len(best_models)*100:.1f}%")
print(f"  ⏱️ 总分析时间: {total_time:.1f}秒 ({total_time/60:.1f}分钟)")

print(f"\n🔧 主要修复:")
print(f"  ✓ 决策图数据累积值计算错误修复")
print(f"    - 问题：Step_0_Cumulative 和 Step_1_Cumulative 显示相同值")
print(f"    - 修复：确保expected_value为标量，正确计算累积SHAP值")
print(f"    - 验证：添加累积值计算验证机制")
print(f"  ✓ 保持概要图数据宽格式（便于Origin使用）")

print(f"\n📁 输出结构:")
print(f"  主目录: {shap_output_dir}")
print(f"  ├── SHAP_Analysis_Summary_Fixed.xlsx (分析结果汇总)")
print(f"  ├── Directory_Index_Fixed.xlsx (目录索引)")
print(f"  ├── Multi_Model_SHAP_Analysis_Report_Fixed.txt (总体报告)")
for i, result in enumerate(all_results):
    if result['success']:
        dir_name = os.path.basename(result['output_dir'])
        print(f"  ├── {dir_name}/ (模型 {i+1})")
        print(f"  │   ├── 可视化文件 ({len(result['visualization_files'])}个PNG)")
        print(f"  │   ├── 数据文件 ({len(result['data_files'])}个XLSX)【决策图数据已修复】")
        print(f"  │   └── model_info.txt")

if successful_analyses > 0:
    print(f"\n🎯 成功分析的模型:")
    for i, result in enumerate(all_results):
        if result['success']:
            model_info = result['model_info']
            advanced_mark = "★" if model_info['Is_Advanced_Method'] else " "
            focus_mark = "🎯" if model_info['Is_Focus_Combination'] else " "
            
            print(f"  {i+1:2d}.{advanced_mark}{focus_mark} {result['model_key']}")
            print(f"      方法: {model_info['Method']}, 模型: {model_info['ML_Model']}, K={model_info['K']}")
            print(f"      Test R²: {model_info['Test_R2']:.4f}, Test MAE: {model_info['Test_MAE']:.4f}")
            print(f"      输出: {os.path.basename(result['output_dir'])}")
            print(f"      文件: {len(result['visualization_files'])}个可视化 + {len(result['data_files'])}个数据文件")

if failed_analyses > 0:
    print(f"\n❌ 失败分析的模型:")
    for i, result in enumerate(all_results):
        if not result['success']:
            print(f"  {i+1:2d}. {result['model_key']}")
            print(f"      错误: {result['error']}")

print(f"\n💡 Origin用户重点关注（修复版）:")
print(f"  📊 使用 shap_summary_beeswarm_data.xlsx (宽格式，便于Origin处理)")
print(f"    - 每行一个样本，每列一个特征的相关信息")
print(f"    - 包含特征值、SHAP值、标准化值")
print(f"    - 直接复制粘贴到Origin即可进行可视化")
print(f"  📈 决策图数据（已修复）:")
print(f"    - shap_decision_plot_data_all_samples.xlsx (推荐)")
print(f"    - 累积值计算已修复：Step_1_Cumulative = Base_Value + Step_1_SHAP")
print(f"    - 可以正确绘制决策图轨迹")
print(f"  📊 其他推荐文件:")
print(f"    - shap_feature_importance.xlsx (特征重要性)")
print(f"    - shap_waterfall_data_all_samples.xlsx (瀑布图数据)")
print(f"    - shap_statistical_analysis.xlsx (统计分析)")

print(f"\n✅ 多模型SHAP分析程序执行完成（修复版）!")
print(f"📁 所有结果已保存到: {shap_output_dir}")
print(f"🕒 完成时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"\n🔍 现在每个模型都有修复的SHAP分析内容:")
print(f"  - 6种可视化图表（概要图、蜂群图、条形图、热图、瀑布图、决策图）")
print(f"  - 11个数据文件（包括修复的决策图数据）")
print(f"  - 完整的统计分析和文件索引")
print(f"  - 详细的模型信息和配置")
print(f"✨ 特别修复了决策图数据累积值计算问题，现在可以正确用于Origin可视化！")

=== 开始多模型SHAP分析（修复版）===
输出目录: D:\博一下\manuscript5-变Si组\SHAP分析8.1.3
分析时间: 2025-08-02 21:22:44

=== 加载原始数据 ===
✓ 成功加载数据: (221, 104)

=== 加载所有最佳模型相关数据 ===
✓ 成功加载所有保存的数据
✓ 找到 11 个最佳模型

=== 开始多模型SHAP分析（修复版）===
总计模型数量: 11

[1/11] 分析模型: PCC_k=5_GradientBoosting
  方法: PCC, 模型: GradientBoosting, K=5
  Test R²: 0.9318, Test MAE: 2.0637
  准备模型数据: PCC_k=5_GradientBoosting
    特征: ['mean_C4 enthalpy melting', 'solubility_limit_factor', 'mean_S10 Lattice Constants a', 'Λ', 'Ω']
    特征索引: [10 33 12  8  7]
    模型输入数据形状: (221, 5)
    使用数据类型: 原始数据
    模型预测成功: [4.6582, 61.5129]
  创建SHAP解释器...
    ✓ TreeExplainer成功
    SHAP值形状: (221, 5)
  创建完整可视化...
    创建完整SHAP可视化图表...
      ✓ SHAP概要图
      ✓ SHAP蜂群图
      ✓ SHAP条形图
      ✓ SHAP热图
      创建SHAP瀑布图...
      ✓ SHAP瀑布图 (10个样本)
      创建SHAP决策图...
      ✓ SHAP决策图
  保存完整数据...
    保存完整SHAP分析数据...
      ✓ 特征重要性数据
      创建概要图/蜂群图数据（宽格式）...
      ✓ 概要图/蜂群图数据（宽格式，便于Origin处理）
      ✓ 概要图/蜂群图数据（长格式备份）
      ✓ 热图数据
      使用基准值: 12.081340271493211
      ✓ 瀑布图数据 (前20个样本)


  0%|          | 0/221 [00:00<?, ?it/s]

    ✓ KernelExplainer成功
    SHAP值形状: (221, 4)
  创建完整可视化...
    创建完整SHAP可视化图表...
      ✓ SHAP概要图
      ✓ SHAP蜂群图
      ✓ SHAP条形图
      ✓ SHAP热图
      创建SHAP瀑布图...
      ✓ SHAP瀑布图 (10个样本)
      创建SHAP决策图...
      ✓ SHAP决策图
  保存完整数据...
    保存完整SHAP分析数据...
      ✓ 特征重要性数据
      创建概要图/蜂群图数据（宽格式）...
      ✓ 概要图/蜂群图数据（宽格式，便于Origin处理）
      ✓ 概要图/蜂群图数据（长格式备份）
      ✓ 热图数据
      使用基准值: 10.912167566666666
      ✓ 瀑布图数据 (前20个样本)
      正在保存所有 221 个样本的瀑布图数据...
      ✓ 瀑布图数据 (所有 221 个样本)
      决策图基准值（随机样本）: 10.912167566666666
      验证决策图数据计算（前5个样本）:
        样本 89: Base=10.912168, Step1_计算=8.856226, Step1_实际=8.856226, 匹配=True
        样本 197: Base=10.912168, Step1_计算=8.699535, Step1_实际=8.699535, 匹配=True
        样本 3: Base=10.912168, Step1_计算=11.114919, Step1_实际=11.114919, 匹配=True
        样本 188: Base=10.912168, Step1_计算=9.560951, Step1_实际=9.560951, 匹配=True
        样本 107: Base=10.912168, Step1_计算=11.606472, Step1_实际=11.606472, 匹配=True
      ✓ 决策图数据 (随机100个样本)
      正在保存所有 221 个样本的决策图数据...
      决策图基准值